In [3]:
import torch
# torch.cuda.is_available()
# torch.cuda.device_count()
# torch.cuda.current_device()
# torch.cuda.device(0)
# torch.cuda.get_device_name(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import numpy as np
import csv
from tqdm import tqdm
import os

# Load the model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

# Read audio file paths
file_path = './path_train.txt'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Define output path for the CSV file
csv_path = 'X_Train.npy'

# Create a CSV writer
with open(csv_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Process each audio file and extract features
    for audio_file in tqdm(audio_files):
        audio_file = audio_file.strip()
        audio_input, sample_rate = sf.read(audio_file)

        # Preprocess the audio input
        inputs = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt")

        # Extract features from the last hidden layer
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_state = outputs.last_hidden_state.squeeze().numpy()

        # Flatten the features for CSV
        flattened_features = last_hidden_state.flatten()

        # Extract the speaker ID from the file path
        speaker_id = os.path.splitext(os.path.basename(audio_file))[0]

        # Write the features to the CSV file
        csv_writer.writerow([speaker_id] + flattened_features.tolist())


/home/mojo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import numpy as np
import csv
from tqdm import tqdm
import os
import torch

# Check CUDA availability and set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self").to(device)

# Read audio file paths
file_path = './path_train.txt'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Define output path for the CSV file
csv_path = 'X_Train.npy'

# Create a CSV writer
with open(csv_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Process each audio file and extract features
    for audio_file in tqdm(audio_files):
        audio_file = audio_file.strip()
        audio_input, sample_rate = sf.read(audio_file)

        # Preprocess the audio input and move it to the device
        inputs = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").to(device)

        # Extract features from the last hidden layer with torch.no_grad
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_state = outputs.last_hidden_state.squeeze()  # No need for numpy conversion on GPU

        # Flatten the features for CSV
        flattened_features = last_hidden_state.flatten().cpu().numpy()  # Move data back to CPU for manipulation

        # Clear GPU memory allocated for this iteration
        del inputs, outputs, last_hidden_state
        torch.cuda.empty_cache()  # Explicitly request memory deallocation

        # Extract the speaker ID from the file path
        speaker_id = os.path.splitext(os.path.basename(audio_file))[0]

        # Write the features to the CSV file
        csv_writer.writerow([speaker_id] + flattened_features.tolist())


/home/mojo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mojo/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This 

AttributeError: 

In [2]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import soundfile as sf
import numpy as np
import csv
from tqdm import tqdm
import os
import torch

# Check CUDA availability and set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self").to(device)

# Read audio file paths
file_path = './path_train.txt'
with open(file_path, 'r') as file:
    audio_files = file.readlines()

# Define output path for the CSV file
csv_path = 'X_Train.npy'

# Create a CSV writer
with open(csv_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Process each audio file with batch size 1 (individual processing)
    for audio_file in tqdm(audio_files):
        audio_file = audio_file.strip()
        audio_input, sample_rate = sf.read(audio_file)

        # Preprocess the audio input and move it to the device
        inputs = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").to(device)

        # Check if `inputs` is a tensor before applying `unsqueeze` (if necessary)
        if isinstance(inputs, torch.Tensor):
            inputs = inputs.unsqueeze(0)  # Add batch dimension only if it's a tensor

        # Extract features from the last hidden layer with torch.no_grad
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_state = outputs.last_hidden_state.squeeze()  # No need for numpy conversion on GPU

        # Flatten the features for CSV
        flattened_features = last_hidden_state.flatten().cpu().numpy()  # Move data back to CPU

        # Clear GPU memory after each audio
        del inputs, outputs, last_hidden_state
        torch.cuda.empty_cache()

        # Extract the speaker ID from the file path
        speaker_id = os.path.splitext(os.path.basename(audio_file))[0]

        # Write the features to the CSV file
        csv_writer.writerow([speaker_id] + flattened_features.tolist())


/home/mojo/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mojo/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This 

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.84 GiB. GPU 